In [146]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [147]:
def load_xls_report(filename):
    data = pd.read_excel(filename, header=None)
    start_index = data[data.iloc[:, 0]=='Transacciones'].index[0]
    end_index = data.iloc[start_index:, 0].isna().idxmax()
    columns = data.iloc[start_index+1].tolist()
    data.columns = columns
    balance = data['Beneficio'].iloc[start_index+2]
    data = data.iloc[start_index+3:end_index-1,:]
    resultados = []
    for i in range(0, len(data) - 1, 2):
        fila_in = data.iloc[i]
        fila_out = data.iloc[i + 1]
        if fila_in['Dirección'] == 'in' and fila_out['Dirección'] == 'out':
            precio_entrada = fila_in['Precio']
            precio_salida = fila_out['Precio']
            beneficio = fila_out['Beneficio'] - abs(fila_in['Comisión']) - abs(fila_out['Comisión'])
            balance += beneficio
            resultados.append({
                'Fecha': fila_in['Fecha/Hora'],
                'Símbolo': fila_in['Símbolo'],
                'Precio Entrada': precio_entrada,
                'Precio Salida': precio_salida,
                'Volumen': fila_in['Volumen '],
                'Beneficio': beneficio,
                'Balance': balance
            })
    return pd.DataFrame(resultados)

In [148]:
filename='ReportTester-4000009439.xlsx'
df = load_xls_report(filename)

In [149]:
# Statistics
total_trades = df.shape[0]
profit_trades = df[df['Beneficio']>0].shape[0]
profit_avg = df[df['Beneficio']>0]['Beneficio'].mean()
loss_trades = df[df['Beneficio']<0].shape[0]
loss_avg = df[df['Beneficio']<0]['Beneficio'].mean()
beneficio_neto  = (profit_avg*profit_trades)+(loss_avg*loss_trades)
total_trades = profit_trades + loss_trades
expected_profit = beneficio_neto / total_trades
profit_avg_points = abs(df[df['Beneficio']>0]['Precio Entrada'] - df[df['Beneficio']>0]['Precio Salida']).mean()
loss_avg_points = abs(df[df['Beneficio']<0]['Precio Entrada'] - df[df['Beneficio']<0]['Precio Salida']).mean()
punto_equilibrio = abs(loss_avg)/(abs(loss_avg)+profit_avg)
win_rates = profit_trades/total_trades

In [150]:
print(f"Número de operaciones totales: {total_trades}")
print(f"Número de operaciones ganadoras: {profit_trades}")
print(f"Número de operaciones perdedoras: {loss_trades}")
print(f"Media de puntos en operaciones ganadoras {profit_avg_points:.2f} puntos")
print(f"Media de puntos en operaciones perdedoras {loss_avg_points:.2f} puntos")
print(f"Ganancia media: {profit_avg:.2f}€")
print(f"Pérdida media: {loss_avg:.2f}€")
print(f"Rentabilidad esperada: {expected_profit:.4f}€")
print(f"Porcentaje de aciertos actual: {win_rates*100.0:.2f}%")
print(f"Porcentaje de aciertos para equilibrio: {punto_equilibrio*100.0:.2f}%")

Número de operaciones totales: 13910
Número de operaciones ganadoras: 10446
Número de operaciones perdedoras: 3464
Media de puntos en operaciones ganadoras 11.08 puntos
Media de puntos en operaciones perdedoras 19.12 puntos
Ganancia media: 9597.65€
Pérdida media: -17873.90€
Rentabilidad esperada: 2756.4247€
Porcentaje de aciertos actual: 75.10%
Porcentaje de aciertos para equilibrio: 65.06%


In [151]:
df

,Fecha,Símbolo,Precio Entrada,Precio Salida,Volumen,Beneficio,Balance
0,2022.01.03 03:30:00,GDAXI,15920.8,15880.1,0.08,32.12,4032.12
1,2022.01.03 04:33:40,GDAXI,15880.3,15875.6,0.08,3.32,4035.44
2,2022.01.03 05:50:40,GDAXI,15875.3,15873.6,0.08,0.92,4036.36
3,2022.01.03 07:01:40,GDAXI,15883.8,15873.6,0.08,7.72,4044.08
4,2022.01.03 07:42:00,GDAXI,15882.3,15878.6,0.08,2.52,4046.60
...,...,...,...,...,...,...,...
13905,2024.10.02 17:19:40,GDAXI,19133.0,19105.7,100,26750.00,38304168.04
13906,2024.10.02 17:49:40,GDAXI,19166.0,19132.7,100,32750.00,38336918.04
13907,2024.10.02 18:35:40,GDAXI,19170.7,19175.7,100,-5550.00,38331368.04
13908,2024.10.02 20:28:40,GDAXI,19160.0,19146.7,100,12750.00,38344118.04
